<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train7_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
import pandas as pd

In [38]:

train7 = pd.read_csv(r'/content/train_data7.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [39]:

train7['Target'] = train7['Target'].replace(['Others'],'Invalid')


In [40]:
train7['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [41]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train7_Xtfidf = tfidf_vect.fit_transform(train7['Sentence'])


In [43]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [44]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [45]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [46]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [47]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [48]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [49]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train7_Xtfidf,train7['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6703539823008849
                  precision    recall  f1-score   support

        Analysis       0.52      0.38      0.44       130
      Conclusion       0.73      0.45      0.56        49
           Facts       0.73      0.91      0.81       475
         Invalid       0.51      0.45      0.47       128
           Issue       0.57      0.45      0.51        53
Rule/Law/Holding       0.61      0.28      0.38        69

        accuracy                           0.67       904
       macro avg       0.61      0.49      0.53       904
    weighted avg       0.65      0.67      0.65       904



In [50]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [51]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 3]
[0.9970934431043723, 0.9999197126069467, 0.9981141476642019, 0.9971757798997778, 0.9837891572371756, 0.9846723445659961, 0.9986376784953834, 0.9940783172014244, 0.9958685443536919, 0.9977829739280509, 0.9960665786024874, 0.9993337906827948, 0.9998437397047395, 0.9913013442148534, 0.9923272368927805, 0.995846979128243, 0.9956620146063896, 0.9958478461411323, 0.9994359792921185, 0.9997163106085732, 0.980930809258888, 0.9886314740951406, 0.995897614497697, 0.9957569889940614, 0.9913195883564624, 0.9972764486496017, 0.995298117014481, 0.9989168750312039, 0.9970233537814784, 0.9947780467288486, 0.9968642841435974, 0.988729738825123, 0.9822582682602777, 0.9908668783135024, 0.9999120453846743, 0.9869867499402792, 0.9902192769011645, 0.9884330508307505, 0.9944699496351201, 0.9938077348941193, 0.999423357900398, 0.9993452400357089, 0.9972770304437175]
[4, 5, 11, 12, 

In [52]:
unlabel_1=unlabel_1.loc[[4, 5, 11, 12, 14, 17, 18, 19, 21, 23, 24, 29, 31, 35, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 55, 56, 60, 62, 68, 69, 70, 72, 74, 75, 76, 78, 79, 84, 87, 89, 94, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts
12,1102,shown vine street drug storeâs supply narcotic...,Facts
14,2901,testimony armstrong showing prior transactions...,Facts


In [53]:
frame_1 = [train7,unlabel_1]
train7_1 = pd.concat(frame_1)
len(train7_1)

3519

In [54]:
x_train_1 = tfidf_vect.transform(train7_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train7_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6676136363636364
                  precision    recall  f1-score   support

        Analysis       0.49      0.46      0.47       102
      Conclusion       0.69      0.46      0.55        39
           Facts       0.75      0.89      0.81       355
         Invalid       0.53      0.52      0.53       110
           Issue       0.61      0.44      0.51        39
Rule/Law/Holding       0.64      0.24      0.35        59

        accuracy                           0.67       704
       macro avg       0.62      0.50      0.54       704
    weighted avg       0.66      0.67      0.65       704



In [55]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 2, 2, 2, 2, 2, 2, 2]
[0.9981949789929674, 0.9992783958363088, 0.9891350419840552, 0.9979205286971928, 0.9974708527568603, 0.9996907378879648, 0.9999839822504522, 0.9864692962059467, 0.9902486468238721, 0.9989127381206698, 0.9939188873833417, 0.9934443165329914, 0.9993394531377824, 0.9832914958869313, 0.9900995927870258, 0.9995916930170484, 0.999973343635489, 0.9995824477663788, 0.9996609275081806, 0.9876904425797657, 0.9995771271826895, 0.9864312158293663, 0.9949675232568839, 0.9996476763502771, 0.9994983783174612, 0.9998344335293485, 0.9994926577426558, 0.9959629899695621, 0.9906076144610465, 0.9985828987904154, 0.9805203214300778, 0.9958654482572254, 0.9843522901844778, 0.9950111821025832, 0.9966433419190678, 0.9933284748063289, 0.9995324397915647, 0.9975864428735522, 0.9991716958122299, 0.9919999541767484, 0.9929496366402373]
[0, 2, 3, 4, 7, 8, 12, 13, 14, 20, 21, 24, 29, 31, 34, 3

In [56]:
unlabel_2 = unlabel_2.loc[[0, 2, 3, 4, 7, 8, 12, 13, 14, 20, 21, 24, 29, 31, 34, 36, 37, 39, 42, 44, 49, 50, 55, 59, 63, 64, 69, 70, 71, 75, 77, 79, 82, 83, 85, 86, 88, 90, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [57]:
frame_2 = [train7_1,unlabel_2]
train7_2 = pd.concat(frame_2)
len(train7_2)

3560

In [58]:
x_train_2 = tfidf_vect.transform(train7_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train7_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6652806652806653
                  precision    recall  f1-score   support

        Analysis       0.46      0.45      0.45       136
      Conclusion       0.74      0.45      0.56        51
           Facts       0.73      0.90      0.81       489
         Invalid       0.56      0.46      0.50       168
           Issue       0.66      0.37      0.47        57
Rule/Law/Holding       0.66      0.31      0.42        61

        accuracy                           0.67       962
       macro avg       0.63      0.49      0.54       962
    weighted avg       0.65      0.67      0.65       962



In [59]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2]
[0.9989796375833142, 0.9982564838194966, 0.9940543412256908, 0.9926940948825362, 0.9997395581666433, 0.9998345009767762, 0.9994696672892899, 0.9846336245636429, 0.9997974231378517, 0.9920772446936964, 0.9996209343307446, 0.9993922120786826, 0.9981576094802693, 0.9863073587359198, 0.9984229796826437, 0.9987811599130031, 0.9994953112010126, 0.9979813043612646, 0.9905855765768632, 0.9989300202632518, 0.996466235067832, 0.9838248016967815, 0.9898137362868455, 0.995625953866021, 0.9993685932411108, 0.997271484533643, 0.9997849662310317, 0.9922694200139004, 0.9915240618297919, 0.9998091656543444, 0.9961427188007381, 0.9977551683971649, 0.9986245140966407, 0.989605541255206, 0.9967748105936571, 0.9827674981624807, 0.9873055017380852, 0.9995907398643603, 0.9923630052352326]
[0, 2, 3, 4, 7, 8, 12, 18, 20, 26, 27, 29, 30, 34, 35, 36, 43, 46, 48, 52, 53, 54, 56, 57, 63, 64, 65, 66

In [60]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 7, 8, 12, 18, 20, 26, 27, 29, 30, 34, 35, 36, 43, 46, 48, 52, 53, 54, 56, 57, 63, 64, 65, 66, 67, 68, 70, 72, 75, 79, 80, 85, 87, 89, 97],:]
frame_3 = [train7_2,unlabel_3]
train7_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train7_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train7_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6611111111111111
                  precision    recall  f1-score   support

        Analysis       0.44      0.39      0.41       101
      Conclusion       0.72      0.47      0.57        45
           Facts       0.74      0.89      0.81       374
         Invalid       0.53      0.47      0.50       116
           Issue       0.63      0.41      0.50        41
Rule/Law/Holding       0.52      0.28      0.36        43

        accuracy                           0.66       720
       macro avg       0.60      0.48      0.52       720
    weighted avg       0.64      0.66      0.64       720



In [61]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2]
[0.9882964635922495, 0.9969883245781522, 0.9966797393389801, 0.9957037973776484, 0.9972793755401209, 0.9871274262679058, 0.9999243703956074, 0.9996843578557985, 0.9846328232375181, 0.9997288280451339, 0.9801308203248541, 0.9980533452308897, 0.9984555377275907, 0.9986633134243469, 0.9996865518647831, 0.9981424106870982, 0.9984244076722768, 0.9983537065038159, 0.9989154251643406, 0.9964927039318083, 0.9997301381031005, 0.9965948334688975, 0.9883714512699369, 0.9937543292358448, 0.9971707268459256, 0.9901944765720878, 0.9989538731939263, 0.9878593758393532, 0.997467058252245, 0.9974326860550496, 0.9947675567397385, 0.9991920432310194, 0.9890105860660222, 0.9995253583182917, 0.999128507959653, 0.9999836265449186, 0.9857244633083696, 0.9970608372280125, 0.9976174926221054, 0.9969509890012899, 0.9993085319385022, 0.9944384645634367, 0.9989664419882264]
[0, 2, 4, 6

In [62]:
unlabel_4=unlabel_4.loc[[0, 2, 4, 6, 7, 8, 10, 15, 17, 18, 21, 22, 27, 28, 29, 31, 32, 34, 37, 38, 40, 42, 46, 49, 50, 55, 56, 57, 58, 60, 66, 67, 69, 70, 71, 75, 76, 82, 85, 87, 93, 97, 99],:]

In [63]:
frame_4 = [train7_3,unlabel_4]
train7_4 = pd.concat(frame_4)
len(train7_4)
x_train_4 = tfidf_vect.transform(train7_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train7_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6808943089430894
                  precision    recall  f1-score   support

        Analysis       0.51      0.43      0.47       139
      Conclusion       0.59      0.42      0.49        53
           Facts       0.77      0.88      0.82       517
         Invalid       0.53      0.50      0.51       147
           Issue       0.65      0.54      0.59        67
Rule/Law/Holding       0.55      0.36      0.44        61

        accuracy                           0.68       984
       macro avg       0.60      0.52      0.55       984
    weighted avg       0.66      0.68      0.67       984



In [64]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0]
[0.9992318136049966, 0.9862615321594617, 0.9975233301988031, 0.9977203089585468, 0.9955877131649346, 0.9977809666333171, 0.9947322052571946, 0.9984679244721815, 0.9994127295234145, 0.9877291155192128, 0.9868783166620213, 0.985293635007461, 0.9997320784531266, 0.9974563377399605, 0.9849199662887648, 0.9879295877495679, 0.9993896154673589, 0.9913608381759875, 0.9857398345823468, 0.9984937823685749, 0.9890291283871598, 0.9989225397954686, 0.9889666310946383, 0.9801812405759669, 0.9915222551915048, 0.9874693988943192, 0.9892489962229922, 0.989146853361643, 0.9943138100869676, 0.9997813278900614, 0.9990318128475434, 0.9881710114368256, 0.9987298083596468, 0.998137973845987, 0.9909655093073819, 0.9956049813608632, 0.995027217170981, 0.9904725379937884]
[3, 4, 7, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 50, 52, 53, 56, 59, 60, 63, 65, 66, 67, 71, 73, 75, 77, 78, 82, 86, 92, 94

In [65]:
unlabel_5 = unlabel_5.loc[[3, 4, 7, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 50, 52, 53, 56, 59, 60, 63, 65, 66, 67, 71, 73, 75, 77, 78, 82, 86, 92, 94, 95, 97, 99, 103, 110],:]

In [66]:
frame_5 = [train7_4,unlabel_5]
train7_5 = pd.concat(frame_5)
len(train7_5)
x_train_5 = tfidf_vect.transform(train7_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train7_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.676056338028169
                  precision    recall  f1-score   support

        Analysis       0.48      0.41      0.44       140
      Conclusion       0.59      0.50      0.54        54
           Facts       0.78      0.88      0.83       528
         Invalid       0.49      0.50      0.49       157
           Issue       0.63      0.47      0.54        47
Rule/Law/Holding       0.56      0.37      0.44        68

        accuracy                           0.68       994
       macro avg       0.59      0.52      0.55       994
    weighted avg       0.66      0.68      0.66       994



In [67]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6660268714011516


In [68]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.6091635839177271, 0.5626270799747256, 0.5777461565985674, None)
